In [34]:
import yfinance as yf
import pandas as pd
import numpy as np
import backtest as bt

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from itertools import combinations

In [35]:
tickers = ['ABBV', 'CB']
timeframe = '5y'
start_cash = 10_000
window = 200

In [36]:
df=yf.download(tickers, period=timeframe)

[*********************100%***********************]  2 of 2 completed


In [37]:
def log_return(df):
    log_df = np.log(df['Close']/df['Close'].shift(1))
    log_df.columns = pd.MultiIndex.from_product([['Log Return'], log_df.columns])
    return log_df
log_return(df)

Log Return          
Ticker           ABBV        CB
Date                           
2020-08-12        NaN       NaN
2020-08-13  -0.008728  0.007037
2020-08-14   0.004111 -0.008112
2020-08-17   0.013374 -0.033600
2020-08-18  -0.007292 -0.006058
...               ...       ...
2025-08-06  -0.011397  0.007258
2025-08-07   0.012957 -0.001188
2025-08-08  -0.004082  0.007141
2025-08-11   0.002975 -0.000885
2025-08-12  -0.004541 -0.003771

[1256 rows x 2 columns]

In [38]:
def lin_reg(df,tickers,window):
    slope_col = [np.nan] * window
    intercept_col = [np.nan] * window
    for i in range(window, len(df)):
        data = df.iloc[i-window:i]
        stock1 = data[tickers[0]].values
        stock2 = data[tickers[1]].values
        stock1 = sm.add_constant(stock1)
        model = sm.OLS(stock2,stock1).fit()
        slope_col.append(model.params[1])
        intercept_col.append(model.params[0])
    return pd.DataFrame({'Slope': slope_col, 'Intercept': intercept_col}, index=df.index)

In [39]:
def calc_spread(data,tickers,window):
    df=data['Close']
    df[['Slope','Intercept']] = lin_reg(df,tickers,window)[['Slope','Intercept']]
    spread = df[tickers[1]] - (df['Intercept'] + df['Slope']*df[tickers[0]])
    spread_mean = spread.rolling(window).mean()
    spread_std = spread.rolling(window).std()
    z_score = (spread - spread_mean)/spread_std
    return pd.DataFrame({'Spread': spread, 'Spread Mean': spread_mean, 'Spread Std': spread_std, 'Z-Score': z_score})

In [40]:
def close(df):
    return df[['Close']]

In [45]:
df.iloc[395:405]

class Pairs(bt.Strategy):
    def __init__(self, data, cash):
        super().__init__(data, cash)
        self.indicator(close, data)
        self.indicator(calc_spread, data, tickers, window)
    
    def run(self, date_index, row):
        super().run(date_index,row)
        if abs(row['Z-Score']) >= 2:
            self.buy('CB', buy_price=self.cash*(-0.1)*row['Z-Score'])
            self.buy('ABBV', buy_price=self.cash*0.1*row['Z-Score'])
        else:
            self.sell('CB')
            self.sell('ABBV')

# Multiplier for divergence from Z-score


In [46]:
test = Pairs(data=df, cash=10000)
print(test.indicators)
testrun = bt.Backtest(df, test)
testrun.run_backtest(monte_carlo_iterations=100, mc_replacement=False)
testrun.visualize()

            (Log Return, ABBV)  (Log Return, CB)  (Close, ABBV)  (Close, CB)  \
Date                                                                           
2020-08-12                 NaN               NaN      78.096283   119.989906   
2020-08-13           -0.008728          0.007037      77.417595   120.837227   
2020-08-14            0.004111         -0.008112      77.736488   119.860954   
2020-08-17            0.013374         -0.033600      78.783127   115.900589   
2020-08-18           -0.007292         -0.006058      78.210739   115.200623   
...                        ...               ...            ...          ...   
2025-08-06           -0.011397          0.007258     196.300003   269.630005   
2025-08-07            0.012957         -0.001188     198.860001   269.309998   
2025-08-08           -0.004082          0.007141     198.050003   271.239990   
2025-08-11            0.002975         -0.000885     198.639999   271.000000   
2025-08-12           -0.004541         -